Prerequisites:
* astroquery
* vaex
* numpy
* pandas
* signal (optional)
* time (optional)
* datetime (optional)

In [5]:
from astroquery.utils.tap.core import Tap
import vaex
import numpy as np
from datetime import datetime
from time import time
import signal
from astroquery.gaia import Gaia

### utility functions

In [3]:
# progress bar
def progress(percent=0, width=50):
    left = int((width * percent) // 100)
    right = width - left
    
    tags = "#" * left
    spaces = " " * right
    percents = f"{percent:.0f}%"
    
    print("\r[", tags, spaces, "]", percents, sep="", end="", flush=True)

In [4]:
# add timeout, such that sending request again after some period of time
def timeout(func, args=(), kwargs={}, timeout_duration=1, default=None):
    import signal
    from time import time
    from requests import HTTPError
    from time import sleep

    class TimeoutError(Exception):
        pass

    def handler(signum, frame):
        raise TimeoutError()

    # set the timeout handler
    t0 = time()
    signal.signal(signal.SIGALRM, handler) 
    signal.alarm(timeout_duration)
    try:
        result = func(*args, **kwargs)
    except TimeoutError as exc:
        result = default
        t1 = time()
        print("too long, requesting again...")
        print(f"time = {round(t1-t0,2)}s")
    except HTTPError:
        result = default
        t1 = time()
        if(t1-t0 < 1):
            print("service unavailable, sleep for 300s")
            print(f"time = {round(t1-t0,2)}s")
            sleep(300)
            print("continue")
        else:
            print("server not responding, try again")
            print(f"time = {round(t1-t0,2)}s")
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except:
        result = default
        t1 = time()
        print("some error")
        print(f"time = {round(t1-t0,2)}s")
    finally:
        signal.alarm(0)
    
    return result

### defining columns

In [14]:
column_gaia = ["source_id", "pm", "pmra", "pmra_error AS e_pmra", "pmdec", 
           "pmdec_error AS e_pmdec", "parallax", "parallax_error AS e_parallax", "phot_g_mean_mag AS Gmag",	"phot_bp_mean_mag AS BPmag", 
           "phot_rp_mean_mag AS RPmag", "radial_velocity AS rv_gaia",	"radial_velocity_error AS e_rv_gaia",
           "bp_rp", "l AS GLON", "b AS GLAT", "teff_gspphot", "teff_gspphot_lower", "teff_gspphot_upper",
           "logg_gspphot", "logg_gspphot_lower", "logg_gspphot_upper"]

column_astrophysical = ["mh_gspphot", "mh_gspphot_lower", "mh_gspphot_upper", "distance_gspphot", "distance_gspphot_lower", 
                         "distance_gspphot_upper", "ag_gspphot", "ag_gspphot_lower", "ag_gspphot_upper",
                         "mh_gspspec", "mh_gspspec_lower", "mh_gspspec_upper", "alphafe_gspspec", "alphafe_gspspec_lower", 
                         "alphafe_gspspec_upper", "fem_gspspec", "fem_gspspec_lower", "fem_gspspec_upper" ,"spectraltype_esphs"]

column_xmatch_tmass = ["original_ext_source_id AS tmass"]

def appendName(element, name):
    string = element.split(" AS ")
    if(len(string) == 1):
        return f"{name}.\"{element}\""
    else:
        return f"{name}.\"{string[0]}\" AS {string[1]}"

column_gaia = list(map(lambda x: appendName(x, "gdr3"), column_gaia))
column_astrophysical = list(map(lambda x: appendName(x, "astrophysical"), column_astrophysical))
column_xmatch_tmass = list(map(lambda x: appendName(x, "tmass"), column_xmatch_tmass))

columns = column_gaia + column_astrophysical  + column_xmatch_tmass
columns

['gdr3."source_id"',
 'gdr3."pm"',
 'gdr3."pmra"',
 'gdr3."pmra_error" AS e_pmra',
 'gdr3."pmdec"',
 'gdr3."pmdec_error" AS e_pmdec',
 'gdr3."parallax"',
 'gdr3."parallax_error" AS e_parallax',
 'gdr3."phot_g_mean_mag" AS Gmag',
 'gdr3."phot_bp_mean_mag" AS BPmag',
 'gdr3."phot_rp_mean_mag" AS RPmag',
 'gdr3."radial_velocity" AS rv_gaia',
 'gdr3."radial_velocity_error" AS e_rv_gaia',
 'gdr3."bp_rp"',
 'gdr3."l" AS GLON',
 'gdr3."b" AS GLAT',
 'gdr3."teff_gspphot"',
 'gdr3."teff_gspphot_lower"',
 'gdr3."teff_gspphot_upper"',
 'gdr3."logg_gspphot"',
 'gdr3."logg_gspphot_lower"',
 'gdr3."logg_gspphot_upper"',
 'astrophysical."mh_gspphot"',
 'astrophysical."mh_gspphot_lower"',
 'astrophysical."mh_gspphot_upper"',
 'astrophysical."distance_gspphot"',
 'astrophysical."distance_gspphot_lower"',
 'astrophysical."distance_gspphot_upper"',
 'astrophysical."ag_gspphot"',
 'astrophysical."ag_gspphot_lower"',
 'astrophysical."ag_gspphot_upper"',
 'astrophysical."mh_gspspec"',
 'astrophysical."mh_gs

In [11]:
# Join with 2MASS,
tap_tmass = Tap(url="https://irsa.ipac.caltech.edu/TAP/sync")

columns_tmass = ["ra", "dec","j_m", "h_m", "k_m", "designation"]
columns_tmass_names = ["ra", "dec", "Jmag", "Hmag", "Kmag", "designation"]

### Divide into multiple RAs
this strategy is to reduce the size of response

In [8]:
# divide into 360 RAs, depend on preference
ras = np.arange(0,361, 1).astype(int)
ras

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

#### Divide further in DECs direction

In [13]:
decs = np.linspace(-90,90,19).astype(int)
decs

array([-90, -80, -70, -60, -50, -40, -30, -20, -10,   0,  10,  20,  30,
        40,  50,  60,  70,  80,  90])

run in loop

In [16]:
TOP = 50_000 # cap maximum rows for each response, so that the response is not exploding

# ra0 for lower boundry and ra1 for upper boundary
# same with dec0 and dec1
for i, (ra0, ra1) in enumerate(zip(ras[:-1], ras[1:])):
    df_com = [] #initial table
    time0 = time()
    progress(0)
    j = 0
    while j < len(decs) -1:
        dec0 = decs[j]
        dec1 = decs[j+1]
        # query gaia data
        # taking wider ra and dec constrains, because of different epoch with 2MASS
        query_gaia = f"""
        SELECT TOP {TOP} {', '.join(columns)}
        FROM gaiadr3.gaia_source AS gdr3
        LEFT JOIN gaiadr3.astrophysical_parameters AS astrophysical ON astrophysical.source_id = gdr3.source_id
        RIGHT JOIN gaiadr3.tmass_psc_xsc_best_neighbour AS tmass ON tmass.source_id = gdr3.source_id
        WHERE gdr3.ra BETWEEN {ra0-1} AND {ra1+1}
        AND gdr3.dec BETWEEN {dec0-1} AND {dec1+1}
        AND parallax > 0
        AND parallax_error/parallax < 0.15
        AND bp_rp BETWEEN -3 AND 6
        AND gdr3.phot_g_mean_mag BETWEEN 3 AND 21
        AND phot_bp_mean_flux_error/phot_bp_mean_flux < 0.15
        AND phot_rp_mean_flux_error/phot_rp_mean_flux < 0.15
        AND ruwe < 1.4	
        AND phot_bp_rp_excess_factor > 1 + 0.015*gdr3.bp_rp*gdr3.bp_rp
        AND phot_bp_rp_excess_factor < 1.3 + 0.06*gdr3.bp_rp*gdr3.bp_rp
        """
        job_gaia = timeout(Gaia.launch_job, args=(query_gaia,), timeout_duration=120)
        if job_gaia == None: #if failed, try again
            print("fail to fetch gaia")
            print("length = ", len(df_com))
            continue
        result_gaia = job_gaia.get_results()
        df_gaia = vaex.from_pandas(result_gaia.to_pandas())
        # query 2MASS data
        query_tmass = f"""
        SELECT TOP {TOP} {", ".join(columns_tmass)} 
        FROM fp_psc
        WHERE ra BETWEEN {ra0} AND {ra1}
        AND dec BETWEEN {dec0} AND {dec1} 
        AND ph_qual = 'AAA'
        """
        job_tmass = timeout(tap_tmass.launch_job, args=(query_tmass,), timeout_duration=120)
        if job_tmass == None: 
            print("fail to fetch tmass")
            print("length = ", len(df_com))
            continue
        result_tmass = job_tmass.get_results()
        df_tmass = result_tmass.to_pandas()
        df_tmass.columns = columns_tmass_names
        # join
        df_tmass = vaex.from_pandas(df_tmass)
        join = df_tmass.join(df_gaia, left_on="designation", right_on="tmass", how="left", allow_duplication=True)
        join.drop(["designation", "tmass"], inplace=True)
        progress((j+1)/(len(decs)-1)*100)
        if(len(df_com) == 0):
            df_com = join
        else:
            df_com = df_com.concat(join)
        j += 1
        t1 = time()
    time1 = time()  
    df_com.rename("sdss13", "SDSS13")
    df_com.rename("rave6", "RAVE6")
    df_com.export(f"gaia-{ra0:03d}-{ra1:03d}.hdf5", progress=True)
    print(f"{len(df_com)} || {round((time1-time0)/60, 2)}m")
    print(f"{i} saved {ra0}-{ra1} || {datetime.now()}")

Approximately 95 mil rows (23 GB)